In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.optim.optimizer import Optimizer
from torchvision.transforms import ToTensor
from torchvision import datasets, models
from torchvision import transforms
from tqdm import tqdm


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_dir = r'D:\Data\dAiv\train'

%matplotlib inline

In [2]:
class Lion(Optimizer):
    def __init__(self, params, lr=1e-4, betas=(0.9, 0.99), weight_decay=0.0):
        if not 0.0 <= lr:
            raise ValueError('Invalid learning rate: {}'.format(lr))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError('Invalid beta parameter at index 0: {}'.format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError('Invalid beta parameter at index 1: {}'.format(betas[1]))
        defaults = dict(lr=lr, betas=betas, weight_decay=weight_decay)
        super().__init__(params, defaults)
    @torch.no_grad()
    def step(self, closure=None):
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                p.data.mul_(1 - group['lr'] * group['weight_decay'])
                grad = p.grad
                state = self.state[p]
                if len(state) == 0:
                    state['exp_avg'] = torch.zeros_like(p)
                exp_avg = state['exp_avg']
                beta1, beta2 = group['betas']
                update = exp_avg * beta1 + grad * (1 - beta1)
                p.add_(torch.sign(update), alpha=-group['lr'])
                exp_avg.mul_(beta2).add_(grad, alpha=1 - beta2)
        return loss

In [3]:
image_transforms = {
    # 훈련 데이터 증강
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    # 검증 데이터에서는 증강하지 않습니다
    'valid':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

data = {
    'train':
    datasets.ImageFolder(root=data_dir, transform=image_transforms['train']),
    'valid':
    datasets.ImageFolder(root=data_dir, transform=image_transforms['valid']),
}

In [4]:
dataloaders = {
    'train': DataLoader(data['train'], batch_size=64, shuffle=True),
    'valid': DataLoader(data['valid'], batch_size=64, shuffle=True)
}

In [5]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(dataloaders['train'].dataset.classes))

model = model.to(device)

# 옵티마이저와 손실함수. CCE (CrossEntropy 손실함수를 사용), 옵티마이저는 Adam.
loss_func = nn.CrossEntropyLoss()
optimizer = Lion(model.parameters())

c:\Users\User\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\User\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in tqdm(range(num_epochs)):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

    return model

# train the model
trained_model = train_model(model, loss_func, optimizer, num_epochs=25)

# save the trained model
torch.save(trained_model.state_dict(), 'model_weights.pth')

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 0/24
----------
train Loss: 0.1581 Acc: 0.9692


  4%|▍         | 1/25 [02:28<59:12, 148.04s/it]

valid Loss: 0.0256 Acc: 0.9903
Epoch 1/24
----------
train Loss: 0.0372 Acc: 0.9890


  8%|▊         | 2/25 [04:00<44:18, 115.57s/it]

...

valid Loss: 0.0000 Acc: 1.0000
Epoch 23/24
----------
train Loss: 0.0075 Acc: 0.9976


 96%|█████████▌| 24/25 [46:37<01:58, 118.82s/it]

valid Loss: 0.0002 Acc: 1.0000
Epoch 24/24
----------
train Loss: 0.0106 Acc: 0.9972


100%|██████████| 25/25 [48:10<00:00, 115.62s/it]

valid Loss: 0.0001 Acc: 1.0000
